In [1]:
import mxnet as mx

/mnt/D/Ubuntu/package/anaconda3/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [2]:
x = mx.symbol.Variable("x")

In [3]:
args = {"x": mx.nd.array([
    [[1, 2], [4, 5], [2, 4]],
    [[3, 2], [5, 6], [7, 5]]
])}

In [4]:
args['x']


[[[ 1.  2.]
  [ 4.  5.]
  [ 2.  4.]]

 [[ 3.  2.]
  [ 5.  6.]
  [ 7.  5.]]]
<NDArray 2x3x2 @cpu(0)>

In [16]:
y = mx.symbol.swapaxes(x, 1, 2)

In [17]:
y.bind(mx.cpu(), args=args).forward()

[
 [[[ 1.  4.  2.]
   [ 2.  5.  4.]]
 
  [[ 3.  5.  7.]
   [ 2.  6.  5.]]]
 <NDArray 2x2x3 @cpu(0)>]

In [18]:
z = mx.symbol.batch_dot(x, y)

In [19]:
z.bind(mx.cpu(), args=args).forward()

[
 [[[  5.  14.  10.]
   [ 14.  41.  28.]
   [ 10.  28.  20.]]
 
  [[ 13.  27.  31.]
   [ 27.  61.  65.]
   [ 31.  65.  74.]]]
 <NDArray 2x3x3 @cpu(0)>]

In [25]:
net = nn.Sequential()
net.add(FancyMLP())

In [23]:
from mxnet.gluon import nn
from mxnet import nd

class FancyMLP(nn.Block):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)
        with self.name_scope():
            self.dense = nn.Dense(256)
            self.weight = nd.random_uniform(shape=(256,20))

    def forward(self, x):
        x = nd.relu(self.dense(x))
        x = nd.relu(nd.dot(x, self.weight)+1)
        x = nd.relu(self.dense(x))
        return x

In [28]:
class ProductDotMLP(nn.Block):
    
    def __init__(self, **kwargs):
        super(ProductDotMLP, self).__init__(**kwargs)
        
    def forward(self, x):
        y = mx.nd.swapaxes(x, 1, 2)
        x = mx.nd.batch_dot(x, y)
        return x

In [29]:
print(net)

Sequential(
  (0): FancyMLP(
    (dense): Dense(None -> 256, linear)
  )
)


In [48]:
net4 = nn.Sequential()
with net4.name_scope():
    net4.add(ProductDotMLP())
    net4.add(nn.Dense(10))

In [49]:
print(net4)

Sequential(
  (0): ProductDotMLP(
  
  )
  (1): Dense(None -> 10, linear)
)


In [50]:
args

{'x': 
 [[[ 1.  2.]
   [ 4.  5.]
   [ 2.  4.]]
 
  [[ 3.  2.]
   [ 5.  6.]
   [ 7.  5.]]]
 <NDArray 2x3x2 @cpu(0)>}

In [51]:
net4.initialize()
y = net4(args['x'])
y


[[  4.39069605   0.33613688   3.82606506   1.94314289   1.33641803
    2.59323287  -4.62039185   0.73926079   2.09413385  -0.75739247]
 [  9.24400139   1.77892494   7.15589476   4.66695595   3.38488364
    4.21873999 -12.04995918   2.16704655   2.43517303   0.8619194 ]]
<NDArray 2x10 @cpu(0)>

In [46]:
args

{'x': 
 [[[ 1.  2.]
   [ 4.  5.]
   [ 2.  4.]]
 
  [[ 3.  2.]
   [ 5.  6.]
   [ 7.  5.]]]
 <NDArray 2x3x2 @cpu(0)>}


[[[ 1.  4.  2.]
  [ 2.  5.  4.]]

 [[ 3.  5.  7.]
  [ 2.  6.  5.]]]
<NDArray 2x2x3 @cpu(0)>

In [59]:
class ProductDotMLP2(nn.Block):
    
    def __init__(self, **kwargs):
        super(ProductDotMLP2, self).__init__(**kwargs)
        
    def forward(self, x):
        y = mx.nd.swapaxes(x[0], 1, 2)
        r0 = mx.nd.batch_dot(x[0], y)
        z = mx.nd.swapaxes(x[1], 1, 2)
        r1 = mx.nd.batch_dot(x[0], z)
        r = mx.nd.add(r0, r1)
        return r

In [60]:
net4 = nn.Sequential()
with net4.name_scope():
    net4.add(ProductDotMLP2())

In [61]:
y = net4([args['x'],args['x']])

In [62]:
y


[[[  10.   28.   20.]
  [  28.   82.   56.]
  [  20.   56.   40.]]

 [[  26.   54.   62.]
  [  54.  122.  130.]
  [  62.  130.  148.]]]
<NDArray 2x3x3 @cpu(0)>